In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import os, re 
import tensorflow as tf
import glob

from sklearn.model_selection import train_test_split

### I. data import

In [2]:
txt_file_path = os.getenv('HOME')+'/aiffel/fundamental/1005_lstm/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['What do you mean?', 'Oh, oh, oh', 'When you sometimes say yes']


### II. pre-processing

#### stemming

In [3]:
# 입력된 문장을
#     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
#     2. 특수문자 양쪽에 공백을 넣고
#     3. 여러개의 공백은 하나의 공백으로 바꿉니다
#     4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
#     5. 다시 양쪽 공백을 지웁니다
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
# 이 순서로 처리해주면 문제가 되는 상황을 방지할 수 있겠네요!
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

# 이 문장이 어떻게 필터링되는지 확인해 보세요.
print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


In [4]:
# 여기에 정제된 문장을 모을겁니다
corpus = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    
    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인해보죠
corpus[:10]

['<start> what do you mean ? <end>',
 '<start> oh , oh , oh <end>',
 '<start> when you sometimes say yes <end>',
 '<start> but you sometimes say no <end>',
 '<start> what do you mean ? <end>',
 '<start> hey , yeah <end>',
 '<start> when you don t want me to move <end>',
 '<start> but you tell me to go <end>',
 '<start> what do you mean ? <end>',
 '<start> oh <end>']

In [5]:
len(corpus)

175749

#### tokenizing

In [6]:
# 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용합니다
# 더 잘 알기 위해 아래 문서들을 참고하면 좋습니다
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen = 15)  ## 토큰 최대 15개만 
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2   40   47 ...    0    0    0]
 [   2   39    4 ...    0    0    0]
 [   2   46    7 ...    0    0    0]
 ...
 [   2    7  224 ...    0    0    0]
 [  19  144    4 ...   19  882    3]
 [   2    8 3460 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f0cfa8a1110>


* 단어장 수는 12,000, 토큰은 15로 리밋

In [7]:
# tensor 길이 확인
print(tensor[0,])
print("\n-----------------------------\n")
print("tensor length:", len(tensor[0,]))

[  2  40  47   7 243  43   3   0   0   0   0   0   0   0   0]

-----------------------------

tensor length: 15


##### source and target split

In [8]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[  2  40  47   7 243  43   3   0   0   0   0   0   0   0]
[ 40  47   7 243  43   3   0   0   0   0   0   0   0   0]


### III. test, train split

#### dataset object

In [9]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
# 데이터셋에 대해서는 아래 문서를 참고하세요
# 자세히 알아둘수록 도움이 많이 되는 중요한 문서입니다
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

In [10]:
src_train, src_test, trg_train, trg_test = train_test_split(src_input, tgt_input, test_size=0.2)

print('source train :', src_train.shape)
print('target train :', trg_train.shape)
print('source test :', src_test.shape)
print('target test  :', trg_test.shape)

source train : (140599, 14)
target train : (140599, 14)
source test : (35150, 14)
target test  : (35150, 14)


### IV. modelling

In [11]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [12]:
# 데이터셋에서 데이터 한 배치만 불러오는 방법입니다.
# 지금은 동작 원리에 너무 빠져들지 마세요~
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[ 2.42224749e-04, -2.10468817e-04, -3.54580789e-06, ...,
         -1.55135549e-05, -5.63139110e-06,  3.83634724e-05],
        [ 3.37018486e-04,  2.44113180e-05, -1.49645530e-05, ...,
         -5.97169274e-05, -7.23142148e-06,  3.25343012e-06],
        [ 1.84258257e-04,  5.09586476e-04, -6.42737432e-05, ...,
         -1.43040554e-04, -2.09511800e-05,  9.87971871e-05],
        ...,
        [ 2.60280591e-04,  1.64098834e-04,  3.11885437e-04, ...,
          3.79758712e-04,  5.45529008e-04,  4.26881161e-04],
        [-3.28313530e-04,  1.82102303e-04,  3.05830676e-04, ...,
          4.23023186e-04,  6.70404290e-04,  1.04190386e-03],
        [-9.69662738e-04,  1.73703243e-04,  3.14635632e-04, ...,
          4.55701549e-04,  7.72519736e-04,  1.71206193e-03]],

       [[ 2.42224749e-04, -2.10468817e-04, -3.54580789e-06, ...,
         -1.55135549e-05, -5.63139110e-06,  3.83634724e-05],
        [ 4.16495022e-04, -1.38897594e-04,  1

In [13]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  3072256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  12301025  
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


#### training

In [14]:
# optimizer와 loss등은 차차 배웁니다
# 혹시 미리 알고 싶다면 아래 문서를 참고하세요
# https://www.tensorflow.org/api_docs/python/tf/keras/optimizers
# https://www.tensorflow.org/api_docs/python/tf/keras/losses
# 양이 상당히 많은 편이니 지금 보는 것은 추천하지 않습니다
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, 
    reduction= "auto")

model.compile(loss=loss, optimizer=optimizer)
result = model.fit(src_train, trg_train, validation_data =(src_test, trg_test), batch_size=BATCH_SIZE, epochs=10)

Epoch 1/10
550/550 [==============================] - 193s 345ms/step - loss: 4.2816 - val_loss: 3.3350
Epoch 2/10
550/550 [==============================] - 190s 345ms/step - loss: 3.0340 - val_loss: 2.9987
Epoch 4/10
550/550 [==============================] - 188s 342ms/step - loss: 2.8884 - val_loss: 2.8993
Epoch 5/10
550/550 [==============================] - 189s 344ms/step - loss: 2.7545 - val_loss: 2.8277
Epoch 6/10
550/550 [==============================] - 189s 344ms/step - loss: 2.6396 - val_loss: 2.7696
Epoch 7/10
550/550 [==============================] - 188s 341ms/step - loss: 2.5302 - val_loss: 2.7181
Epoch 8/10
550/550 [==============================] - 188s 341ms/step - loss: 2.4284 - val_loss: 2.6784
Epoch 9/10
550/550 [==============================] - 189s 343ms/step - loss: 2.3336 - val_loss: 2.6431
Epoch 10/10
550/550 [==============================] - 188s 342ms/step - loss: 2.2448 - val_loss: 2.6149


### V. text generation

In [17]:
def generate_text(lyricist, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [18]:
generate_text(model, tokenizer, init_sentence="<start> she")

'<start> she s a <unk> <end> '

* 그닥 그럴듯하지 않다  

In [21]:
generate_text(model, tokenizer, init_sentence="<start> i am")

'<start> i am not throwing away my shot <end> '

* 제법 그럴듯하다

#### 회고

- validation loss를 2.2이하로 낮추지 못했다 
- 블랙박스일 뿐인듯한 알고리즘을 쓰는게 무엇에 얼마나 용이한 것인지 모르겠다